In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pprint import pprint
from PIL import Image
import glob
import scipy.misc
import imageio
from skimage.transform import resize
import tensorflow as tf
import keras

import h5py, itertools, collections
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator, DirectoryIterator

In [ ]:
image_size = (256, 256)
batch_size = 32

image_generator = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1, 
        zoom_range=0.2,
        validation_split=0.1,
        rescale = 1/255.0)

train_ds =  DirectoryIterator(
    #"폴더 위치",
    image_generator,
    class_mode='categorical',
    seed=1337,
    target_size=image_size,
    batch_size=batch_size,
    subset = 'training',)

val_ds = DirectoryIterator(
    #"폴더 위치",
    image_generator,
    class_mode='categorical',
    seed=1337,
    target_size=image_size,
    batch_size=batch_size,
    subset = 'validation',
    shuffle=False)
# 케릭터 이름
class_names = list(train_ds.class_indices.keys())

# 케릭터 레이블
num_classes = train_ds.num_classes

In [ ]:
#시각화
plt.figure(figsize=(20, 16))
images = []
labels = []
for itr in train_ds.next():
    for i in range(30):
        if len(images) < 30:
            images.append(itr[i].astype("float32"))
        else:
            labels.append(list(itr[i]).index(1))

for i in range(len(images)):
    ax = plt.subplot(5, 6, i + 1)
    plt.imshow(images[i])
    plt.title(class_names[labels[i]].replace('_',' ') +' ('+str(int(labels[i]))+')')
    plt.axis("off")

In [ ]:
#Keras 몬델 로딩
conv_base = tf.keras.applications.DenseNet121(weights='imagenet', include_top=False)

In [ ]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Activation
from tensorflow.keras.optimizers import Adam
from keras import layers

optimizers = Adam(learning_rate=0.0001)
inputs = keras.Input(shape=image_size + (3,))
x = inputs


x = conv_base(x)
x = GlobalAveragePooling2D()(x)

#x = layers.Flatten()(x)
#x = layers.Dropout(0.7)(x)


x = Dense(1024, activation='relu')(x)
x = layers.Dropout(0.7)(x)
x = Dense(9)(x)
outputs = Activation(activation='softmax')(x)
model = keras.Model(inputs, outputs)


model.compile(optimizer=optimizers,
              loss='CategoricalCrossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
conv_base.trainable = True

In [ ]:
history = model.fit(
  train_ds,
  epochs=50,
  validation_data=val_ds,
  )

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

epochs = [i+1 for i in range(len(history.history['loss']))]
ax.plot(history.history['loss'], label='loss')
ax.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

epochs = [i+1 for i in range(len(history.history['accuracy']))]
ax.plot(history.history['accuracy'], label='acc')
ax.plot(history.history['val_accuracy'], label='val_acc')
plt.legend()
plt.show()